## run request with owslib

In [1]:
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput


In [2]:
#wps = WebProcessingService(url="http://localhost:8094/wps", verbose=False)
wps = WebProcessingService(url="http://birdhouse-lsce.extra.cea.fr:8093/wps", verbose=False)

In [3]:
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract, )

visualisation : 	 Just testing a nice script to visualise some variables
extractpoints : 	 Extract Timeseries for specified coordinates from grid data
segetalflora : 	 Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44
indices_single : 	 This process calculates climate indices based on one single variable.
subset_countries : 	 This process returns only the given polygon from input netCDF files.
eobs_to_cordex : 	 downloades EOBS data in adaped CORDEX format
modelUncertainty : 	 Calculates the ensemble mean and uncertainty mask


In [4]:
in_file1 = 'file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/tas_day_HadGEM3-A-N216_historical_r1i1p1_19600101-19691230.nc'
#in_file1 = 'file:///homel/nhempel/data/NCEP/slp.1948-2014_NA.nc'
#in_file1 ='file:///home/estimr2/yiou/NCEP/slp.1982_NHX.nc'
#in_file1 = 'file:///home/pingu/Public/data/esgf-archive/CORDEX/cordex/output/WAS-44/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/day/tasmax/v20140918/tasmax_WAS-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_day_20010101-20051231.nc'

In [5]:
execute = wps.execute(
    identifier="subset_countries", #indices_clipping",
    inputs=[
    ("resource",in_file1),
      # ("resource",in_file2),
    ("region",'FRA'),
      # ("country",'DEU'),
    ("mosaik",'False'),
           ])

monitorExecution(execute, sleepSecs=5)

print execute.getStatus()

for o in execute.processOutputs:
    print o.reference

XMLSyntaxError: Opening and ending tag mismatch: hr line 5 and body, line 6, column 8

## run sync request with get parameters

In [6]:
sync_req_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=subset_countries" +\
    "&DataInputs=resource={resource};region={region}" +\
    "&storeExecuteResponse=false" +\
    "&status=false"

In [7]:
url=sync_req_url.format(
    resource=in_file1,
    region='FRA')
print url   

http://birdhouse-lsce.extra.cea.fr:8093/wps?request=Execute&service=WPS&version=1.0.0&identifier=subset_countries&DataInputs=resource=file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/tas_day_HadGEM3-A-N216_historical_r1i1p1_19600101-19691230.nc;region=FRA&storeExecuteResponse=false&status=false


In [8]:
import requests
r = requests.get(url)
print r.status_code
print r.text

502
<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.8.0</center>
</body>
</html>



## run async request with get parameters

In [9]:
async_req_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=subset_countries" +\
    "&DataInputs=resource={resource};region={region}" +\
    "&storeExecuteResponse=true" +\
    "&status=true"

In [10]:
url=async_req_url.format(
    resource=in_file1,
    region='FRA')
print url   

http://birdhouse-lsce.extra.cea.fr:8093/wps?request=Execute&service=WPS&version=1.0.0&identifier=subset_countries&DataInputs=resource=file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/tas_day_HadGEM3-A-N216_historical_r1i1p1_19600101-19691230.nc;region=FRA&storeExecuteResponse=true&status=true


In [11]:
r = requests.get(url)
from lxml import etree
from io import BytesIO
tree = etree.parse(BytesIO(r.content))
#print etree.tostring(tree)
status_url = tree.getroot().get("statusLocation")
print status_url

http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/pywps-d2627a22-be01-11e5-b796-29e49ebfeffc.xml


In [13]:
r = requests.get(status_url)
print r.status_code
print r.text

200
<?xml version="1.0" encoding="utf-8"?>
<wps:ExecuteResponse xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsExecute_response.xsd" service="WPS" version="1.0.0" xml:lang="en-CA" serviceInstance="http://birdhouse-lsce.extra.cea.fr:8093/wps?service=WPS&amp;request=GetCapabilities&amp;version=1.0.0" statusLocation="http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/pywps-d2627a22-be01-11e5-b796-29e49ebfeffc.xml">
    <wps:Process wps:processVersion="0.1">
        <ows:Identifier>subset_countries</ows:Identifier>
        <ows:Title>Subset netCDF files</ows:Title>
        <ows:Abstract>This process returns only the given polygon from input netCDF files.</ows:Abstract>
    </wps:Process>
    <wps:Status creationTime="2016-01-18T17:37:58Z">
        <w

In [15]:
r = requests.get(status_url)
print r.status_code
print r.text

200
<?xml version="1.0" encoding="utf-8"?>
<wps:ExecuteResponse xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsExecute_response.xsd" service="WPS" version="1.0.0" xml:lang="en-CA" serviceInstance="http://birdhouse-lsce.extra.cea.fr:8093/wps?service=WPS&amp;request=GetCapabilities&amp;version=1.0.0" statusLocation="http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/pywps-d2627a22-be01-11e5-b796-29e49ebfeffc.xml">
    <wps:Process wps:processVersion="0.1">
        <ows:Identifier>subset_countries</ows:Identifier>
        <ows:Title>Subset netCDF files</ows:Title>
        <ows:Abstract>This process returns only the given polygon from input netCDF files.</ows:Abstract>
    </wps:Process>
    <wps:Status creationTime="2016-01-18T17:39:04Z">
        <w